In [1]:
!pip install pydub

In [1]:
#import library
import numpy as np # Ini buat array

import matplotlib.pyplot as plt
%matplotlib inline

import pydub # Ini buat mbaca mp3
import librosa # Ini buat MFCC

# Ini fungsi buat mbaca mp3
def read(f, normalized=False):
    a = pydub.AudioSegment.from_mp3(f)
    y = np.array(a.get_array_of_samples())
    if a.channels == 2:
        y = y.reshape((-1, 2))
    if normalized:
        return a.frame_rate, np.float32(y) / 2**15
    else:
        return a.frame_rate, y

# baca Suara
sr_howl, x_howl = read('Kucing/howl.mp3')
sr_purr, x_purr = read('Kucing/purring.mp3')
sr_meow, x_meow = read('Kucing/meow.mp3')
sr_mating, x_mating = read('Kucing/Mating Call.mp3')
#x_howl0 = x_howl[:,0].astype(float)
#x_howl1= x_howl[:,1].astype(float)
#x_purr0= x_purr[:,0].astype(float)
#x_purr1= x_purr[:,1].astype(float)
#x_meow0= x_meow[:,0].astype(float)
#x_meow1= x_meow[:,1].astype(float)
#x_mating0= x_mating[:,0].astype(float)
#x_meow1= x_meow[:,1].astype(float)

#disamakan panjangnya
x_howl=x_howl[:3600*sr_howl]
x_purr=x_purr[:3600*sr_purr]
x_meow=x_meow[:3600*sr_meow]
x_mating=x_mating[:3600*sr_mating]

y = []

purrlist=[]
for x in range(0, 3600-20, 20):
  purrlist.append(x_purr[x*sr_purr:(x+20)*sr_purr])
  y.append(0)
    
matinglist=[]
for x in range(0, 3600-20, 20):
  matinglist.append(x_mating[x*sr_mating:(x+20)*sr_mating])
  y.append(1)

meowlist=[]
for x in range(0, 3600-20, 20):
  meowlist.append(x_meow[x*sr_meow:(x+20)*sr_meow])
  y.append(2)

howllist=[]
for x in range(0, 3600-20, 20):
    howllist.append(x_howl[x*sr_howl:(x+20)*sr_howl])
    y.append(3)

#disatukan
X = howllist+purrlist+meowlist+matinglist

import random
C= list(zip(X,y))
random.shuffle(C)
X,y = zip(*C)
X = np.asarray(X)
y = np.asarray(y)

In [2]:
X.shape

(716, 882000, 2)

In [3]:
#CNN -> LSTM
from keras.models import Sequential
from keras.layers import Dense, LSTM, Embedding, Bidirectional, TimeDistributed, Conv1D

model = Sequential()

#model.add(Embedding(kosakata, embed_fitur, input_length=x.shape[1], input_shape=x.shape[1:]))
model.add(Conv1D(4, 15, input_shape=X.shape[1:], strides=15, activation='relu'))
model.add(Conv1D(4, 15, strides=15, activation='relu'))
model.add(Conv1D(4, 15, strides=15, activation='relu'))
model.add(Conv1D(4, 15, strides=15, activation='relu'))
model.add(LSTM(8, return_sequences=True))
model.add(LSTM(8))
model.add(Dense(16, activation='relu'))
model.add(Dense(np.max(y)+1, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 58800, 4)          124       
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 3920, 4)           244       
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 261, 4)            244       
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 17, 4)             244       
_________________________________________________________________
lstm (LSTM)                  (None, 17, 8)             416       
_________________________________________________________________
lstm_1 (LSTM)                (None, 8)                 544       
_________________________________________________________________
dense (Dense)                (None, 16)                1

In [4]:
model.fit(X, y, epochs=40, batch_size=1, validation_split=0.2)

Epoch 1/40
572/572 [==============================] - 32s 52ms/step - loss: 1.2235 - accuracy: 0.4038 - val_loss: 0.8041 - val_accuracy: 0.6181
Epoch 2/40
572/572 [==============================] - 15s 25ms/step - loss: 0.6577 - accuracy: 0.7150 - val_loss: 0.5919 - val_accuracy: 0.7222
Epoch 3/40
572/572 [==============================] - 14s 25ms/step - loss: 0.5960 - accuracy: 0.7378 - val_loss: 0.5929 - val_accuracy: 0.7083
Epoch 4/40
572/572 [==============================] - 20s 35ms/step - loss: 0.5271 - accuracy: 0.7675 - val_loss: 0.4638 - val_accuracy: 0.7917
Epoch 5/40
 14/572 [..............................] - ETA: 22s - loss: 0.2613 - accuracy: 1.0000

KeyboardInterrupt: 

In [5]:
model.save_weights('8-8L4444E40.h5')